# Chatbot com LangGraph

## Configuração

Carregue variáveis de ambiente

In [ ]:
from dotenv import load_dotenv
_ = load_dotenv()

## Modelo

Configure o LangChain para usar o LLM desejado.

In [ ]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model="llama3.2", 
    temperature=0.0, 
    max_tokens=2000,
)

In [ ]:
# from langchain_cerebras import ChatCerebras
# llm = ChatCerebras(
#     model="llama-3.3-70b",
#     temperature=0.0,
#     max_tokens=2000,
# )

In [ ]:
# from langchain_groq import ChatGroq
# llm = ChatGroq(
#     model="qwen-qwq-32b", 
#     temperature=0.0, 
#     max_tokens=2000,
# )

In [ ]:
# from langchain_groq import ChatGroq
# llm = ChatGroq(
#     model="llama3-70b-8192", 
#     temperature=0.0, 
#     max_tokens=2000,
# )

In [ ]:
# from langchain_groq import ChatGroq
# llm = ChatGroq(
#     model="qwen-qwq-32b", 
#     temperature=0.0, 
#     max_tokens=2000,
# )

## Chatbot

Defina o template do prompt de sistema do chatbot

In [ ]:
from datetime import datetime

prompt_template = """Você é uma assistente virtual amistosa e sempre disposta a ajudar.
Seu nome é Ana. Você atende clientes da empresa ACME.

Siga sempre as instruções abaixo:

- APENAS na sua primeira mensagem enviada ao cliente:
  fale o nome do cliente e deseje 'bom dia', 'boa tarde' ou 'boa noite', de acordo com o horário atual.
  Por exemplo: 'Bom dia, <nome do cliente>!'
- Dê respostas concisas.
- Utilize apenas as informações mencionadas na conversa para responder as perguntas.

Informações gerais:

- Horário atual: {current_time}
- Data atual: {current_date}

Informações do cliente:

- Nome: José da Silva
- Localização atual: Campinas, São Paulo
- Comidas preferidas: vegetariana
"""

def make_system_prompt():
    current_time = datetime.now().strftime(r"%H:%M:%S")
    current_date = datetime.now().strftime(r"%A, %Y-%m-%d")
    system_prompt = prompt_template.format(current_time=current_time, current_date=current_date)
    return system_prompt

Código do chatbot usando LangGraph:

In [ ]:
from typing import Annotated

from typing_extensions import TypedDict

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, SystemMessage

from IPython.display import Image, display

class ChatbotState(TypedDict):
    messages: Annotated[list, add_messages]

def chatbot_node(state: ChatbotState):
    prompt = [SystemMessage(make_system_prompt())] + state["messages"]
    response = llm.invoke(prompt)
    return {"messages": [response]}

builder = StateGraph(ChatbotState)
builder.add_node("chatbot", chatbot_node)
builder.set_entry_point("chatbot")
builder.set_finish_point("chatbot")

memory = MemorySaver()

graph = builder.compile(checkpointer=memory)

display(Image(graph.get_graph(xray=True).draw_mermaid_png()))


Código para facilitar o uso do chatbot:

In [ ]:
def chatbot(message: str, session_id="1"):
    config = {"configurable": {"thread_id": session_id}}
    messages = {"messages": [HumanMessage(content=message)]}
    index = -1
    for event in graph.stream(messages, config=config, stream_mode="values"):
        for msg in event["messages"][index:]:
            msg.pretty_print()
            print("\n")
        index = len(event["messages"])

def clear_memory():
    memory.storage.clear()

def show_history(session_id="1"):
    config = {"configurable": {"thread_id": session_id}}
    for msg in graph.get_state(config).values["messages"]:
        msg.pretty_print()
        print("\n")

## Conversando com o chatbot

In [ ]:
chatbot("Olá! Você pode recomendar um restaurante?")

In [ ]:
chatbot("Pensando melhor, hoje é meu aniversário e prefiro ir ao cinema primeiro. "
        "Mas olha, agora estou em São José dos Campos. "
        "Até pensei em ver teatro também. Eu gosto de cinema e teatro.")

In [ ]:
chatbot("Bom nesse caso, preciso sacar dinheiro. Tem algum Banco do Brasil aqui perto? Eu tenho conta no banco do brasil.")

In [ ]:
chatbot("Você sabe em que cidade eu estou agora?")